In [1]:
import config
from config import corpus_path
print(corpus_path.split('/')[-1])

2016-10-20_article_all_normed_nountokenized.txt


In [2]:
import soynlp
from soynlp.utils import DoublespaceLineCorpus

sentences = DoublespaceLineCorpus(corpus_path, iter_sent=True)
len(sentences)

30002

In [3]:
# from wordvec_infer import sents_to_word_contexts_matrix

# x, idx2vocab = sents_to_word_contexts_matrix(sentences)

In [4]:
# vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}

In [5]:
from wordvec_infer import Word2Vec

def my_tokenizer(sent, stopwords={'아이오아이'}):
    words = [word for word in sent.split() if not (word in stopwords)]
    return words

word2vec = Word2Vec(tokenizer=my_tokenizer, beta=1.0)
word2vec.train(sentences)

  - counting word frequency from 30001 sents, mem=0.094 Gb #vocabs = 24906
  - scanning (word, context) pairs from 30001 sents, mem=0.470 Gb
Training PMI ... done
Training SVD ... done


In [6]:
word2vec.most_similar('너무너무너무', 20)

[('신용재', 0.91987635806576),
 ('타이틀곡', 0.914908670023072),
 ('백퍼센트', 0.8990926450997301),
 ('완전체', 0.8921406573384947),
 ('몬스타엑스', 0.8818118556260817),
 ('엠카운트다운', 0.8816437934129916),
 ('안무', 0.8779975011273335),
 ('열창', 0.8743633278054797),
 ('다비치', 0.8663248282954272),
 ('파이터', 0.8645118789327619),
 ('하이포', 0.8587448107377045),
 ('신곡', 0.8550658914947793),
 ('박진영', 0.8530412783291585),
 ('팡파레', 0.8433407143400704),
 ('수록곡', 0.8432095622258016),
 ('곡으로', 0.8402485071092015),
 ('레이디스코드', 0.8324961897393519),
 ('갓세븐', 0.8319702520657125),
 ('중독성', 0.8277036732753654),
 ('자작곡', 0.8247467952278615)]

In [7]:
word2vec.most_similar('아이오아이')

[]

In [8]:
word2vec.wv.shape

(24906, 100)

In [9]:
word2vec._components.shape

(24906, 100)

In [10]:
from wordvec_infer import sents_to_unseen_word_contexts_matrix

X, idx2vocab = sents_to_unseen_word_contexts_matrix(
    sentences, ['아이오아이'], word2vec._vocab2idx)

Create (unseen word, contexts) matrix
  - counting word frequency from 30001 sents, mem=0.366 Gb #vocabs = 24907
  - scanning (word, context) pairs from 30001 sents, mem=0.366 Gb
  - (word, context) matrix was constructed. shape = (24907, 23632)                    
  - done


In [11]:
wv = word2vec.inference_words(X, idx2vocab, )

In [12]:
word2vec.most_similar('너무너무너무')

[('신용재', 0.91987635806576),
 ('타이틀곡', 0.914908670023072),
 ('백퍼센트', 0.8990926450997301),
 ('완전체', 0.8921406573384947),
 ('몬스타엑스', 0.8818118556260817),
 ('엠카운트다운', 0.8816437934129916),
 ('안무', 0.8779975011273335),
 ('열창', 0.8743633278054797),
 ('다비치', 0.8663248282954272),
 ('파이터', 0.8645118789327619)]

In [13]:
word2vec.wv.shape

(24907, 100)

In [14]:
wv = word2vec.inference_words(X, idx2vocab)
word2vec.wv.shape

(24907, 100)

In [15]:
wv = word2vec.inference_words(X, idx2vocab)
word2vec.wv.shape

(24907, 100)

In [53]:
class Word2VecCorpus:
    def __init__(self, path, num_doc=-1, verbose_point=-1):
        self.path = path
        self.num_doc = num_doc
        self.verbose_point = verbose_point

    def __iter__(self):
        vp = self.verbose_point
        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                if vp > 0 and i % vp == 0:
                    print('\riterating corpus ... %d lines' % (i+1), end='', flush=True)
                if self.num_doc > 0 and i >= self.num_doc:
                    break
                doc = doc.strip()
                if not doc:
                    continue
                streams = self._tokenize(i, doc)
                for stream in streams:
                    yield stream
            if vp > 0:
                print('\riterating corpus was done%s' % (' '*20))

    def _tokenize(self, i, doc):
        for sent in doc.split('  '):
            if sent:
                yield sent.split()

class Doc2VecCorpus(Word2VecCorpus):
    def _tokenize(self, i, doc):
        column = doc.split('\t')
        if len(column) == 1:
            labels = ['__doc{}__'.format(i)]
            words = column[0].split()
        else:
            labels = column[0].split()
            words = [word for col in column[1:] for word in col.split()]
        yield labels, words

In [56]:
corpus_path = '/mnt/lovit/works/fastcampus_text_ml/3rd/data/corpus_10days/news/2016-10-20_article_all_normed.txt'
word2vec_corpus = Word2VecCorpus(corpus_path, num_doc=5, verbose_point=1)
doc2vec_corpus = Doc2VecCorpus(corpus_path, num_doc=5)

In [57]:
for doc in doc2vec_corpus:
    print(doc)

(['__doc1__'], ['19', '1990', '52', '1', '22'])
(['__doc2__'], ['오패산터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자들이', '19일', '오후', '서울', '강북구', '오패산', '터널', '인근에서', '사제', '총기를', '발사해', '경찰을', '살해한', '용의자', '성모씨를', '검거하고', '있다', '성씨는', '검거', '당시', '서바이벌', '게임에서', '쓰는', '방탄조끼에', '헬멧까지', '착용한', '상태였다', '독자제공', '영상', '캡처', '연합뉴스', '서울', '연합뉴스', '김은경', '기자', '사제', '총기로', '경찰을', '살해한', '범인', '성모', '46', '씨는', '주도면밀했다', '경찰에', '따르면', '성씨는', '19일', '오후', '강북경찰서', '인근', '부동산', '업소', '밖에서', '부동산업자', '이모', '67', '씨가', '나오기를', '기다렸다', '이씨와는', '평소에도', '말다툼을', '자주', '한', '것으로', '알려졌다', '이씨가', '나와', '걷기', '시작하자', '성씨는', '따라가면서', '미리', '준비해온', '사제', '총기를', '이씨에게', '발사했다', '총알이', '빗나가면서', '이씨는', '도망갔다', '그', '빗나간', '총알은', '지나가던', '행인', '71', '씨의', '배를', '스쳤다', '성씨는', '강북서', '인근', '치킨집까지', '이씨', '뒤를', '쫓으며', '실랑이하다', '쓰러뜨린', '후', '총기와', '함께', '가져온', '망치로', '이씨', '머리를', '때렸다', '이', '과정에서', '오후', '6시', '20분께', '강북구', '번동', '길', '위에서', '사람들이', '싸우고', '있다', '총소리가', '났다', '는', '등의', '신고가', '여러건', '들어왔다',

In [59]:
for sent in word2vec_corpus:
    continue

iterating corpus was done                    
